## Run pipeline templates

These are templates for running the pipline. Note this depends on preprocessed DepMap data. See *run01-preprocess_data.ipynb* for such run.

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from ceres_infer.session import workflow
from ceres_infer.models import model_infer_iter_ens

/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.


In [3]:
import logging
logging.basicConfig(level=logging.INFO)

In [4]:
# Parameters
params = {
    # directories
    'outdir_run': '../out/20.0926 feat/reg_rf_boruta/', # output dir for the run
    'outdir_modtmp': '../out/20.0926 feat/reg_rf_boruta/model_perf/', # intermediate files for each model
    'indir_dmdata_Q3': '../out/20.0925 proc_data/gene_effect/dm_data.pkl', # pickled preprocessed DepMap Q3 data
    'indir_dmdata_external': '../out/20.0925 proc_data/gene_effect/dm_data_Q4.pkl', # pickled external validation data
    'indir_genesets': '../data/gene_sets/',
    'indir_landmarks': None, # csv file of landmarks [default: None]

    # notes
    'session_notes': 'regression model, with random forest (iterative) and boruta feature selection; \
    run on selective dependent genes (CERES std > 0.25 and CERES range > 0.6)',

    # data
    'ext_data_name': 'Sanger', # name of external validation dataset
    'opt_scale_data': True, # scale input data True/False
    'opt_scale_data_types': '\[(?:RNA-seq|CN)\]', # data source types to scale; in regexp
    'model_data_source': ['CERES','RNA-seq','CN','Mut','Lineage'],
    'anlyz_set_topN': 10, # for analysis set how many of the top features to look at
    'perm_null': 1000, # number of samples to get build the null distribution, for corr
    'useGene_dependency': False, # whether to use CERES gene dependency (true) or gene effect (false)
    'scope': ['KRAS','TP53'], # scope for which target genes to run on; list of gene names, or 'all', 'differential'

    # model
    'model_name': 'rf',
    'model_params': {'n_estimators':1000,'max_depth':15,'min_samples_leaf':5,'max_features':'log2'},
    'model_paramsgrid': {},
    'model_pipeline': model_infer_iter_ens,
    'pipeline_params': {},
    
    # pipeline
    'parallelize': False, # parallelize workflow
    'processes': 1, # number of cpu processes to use
    
    # analysis
    'metric_eval': 'score_test',  # metric in model_results to evaluate, e.g. score_test, score_oob
    'thresholds': {'score_rd10': 0.1,  # score of reduced model - threshold for filtering
                   'recall_rd10': 0.95},  # recall of reduced model - threshold for filtering
    'min_gs_size': 4 # minimum gene set size, to be derived
}

In [ ]:
# Run whole pipeline
wf = workflow(params)
wf.run_pipe()

In [ ]:
# Run just the inference
wf = workflow(params)
pipeline = ['load_processed_data', 'infer']
wf.create_pipe(pipeline)
wf.run_pipe()

In [ ]:
# Run analysis, based on pre-existing inference
wf = workflow(params)
pipeline = ['load_processed_data', 'load_model_results', 
            'analyze', 'analyze_filtered', 'derive_genesets', 'run_Rscripts']
wf.create_pipe(pipeline)
wf.run_pipe()

In [ ]:
# Run just the inference
import os
wf = workflow(params)
wf._get_analysis_paths()

In [ ]:
# Analyze specific genes
from ceres_infer.analyses import genBarPlotGene
outdir = os.path.join(params['outdir_run'], gene_specific)
if not os.path.exists(outdir_sub):
    os.makedirs(outdir)

#generate plot of specific gene
genBarPlotGene(model_results, 'CDK4', 'score_oob', 0.5, outdir)
genBarPlotGene(model_results, 'KRAS', 'score_oob', 0.5, outdir)
genBarPlotGene(model_results, 'SOX10', 'score_oob', 0.5,  outdir)